### Creating DataLoader
This assumes that we have the actual folder with the eval train and test separation

We will only load the data from train in this notebook

In [9]:
# import kagglehub
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
import torch.nn as nn
from torchvision.models import efficientnet_b5, EfficientNet_B5_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url
import torch.optim as optim
import torch

In [12]:
path="../input/actual/train"

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

allDataSet = datasets.ImageFolder(root=path, transform=transform)
totalSize = len(allDataSet)
reductionRatio = 0.5
reducedSize = int(totalSize * reductionRatio)
indices = list(range(totalSize))
random.shuffle(indices)
reduced_indices = indices[:reducedSize]
reducedDataset = Subset(allDataSet, reduced_indices)

print(f"Reduced Train: {len(reducedDataset)}")
dataloader = DataLoader(reducedDataset, batch_size=32, shuffle=True)


Reduced Train: 3262


In [ ]:
# Number of classes in your dataset
num_classes = len(reducedDataset.dataset.classes) 
print(num_classes)

5


Basic retraining of models

In [17]:
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

efficientnet_b5(weights=EfficientNet_B5_Weights.IMAGENET1K_V1)
model = efficientnet_b5(weights="DEFAULT")

num_classes = len(reducedDataset.dataset.classes)
print("Number of classes:", num_classes)

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Dropout(p=0.4),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/total:.4f} Accuracy: {acc:.4f}")

Number of classes: 5
Epoch [1/10] Loss: 1.4494 Accuracy: 0.3967
Epoch [2/10] Loss: 1.2482 Accuracy: 0.5129
Epoch [3/10] Loss: 1.1708 Accuracy: 0.5414
Epoch [4/10] Loss: 1.1320 Accuracy: 0.5552
Epoch [5/10] Loss: 1.0926 Accuracy: 0.5671
Epoch [6/10] Loss: 1.0951 Accuracy: 0.5708
Epoch [7/10] Loss: 1.0813 Accuracy: 0.5723
Epoch [8/10] Loss: 1.0491 Accuracy: 0.5861
Epoch [9/10] Loss: 1.0620 Accuracy: 0.5855
Epoch [10/10] Loss: 1.0422 Accuracy: 0.5886


In [18]:
torch.save(model.state_dict(), "../models/efficientnet_b5_finetuned.pth")
